In [1]:
import pandas as pd

INSHEP = pd.read_csv(r'C:\Users\Adnane\Desktop\Radar\ObjectClassificationWithRadar\Training_with_our_features\INSHEP_features.csv')
SPECTROGRAM = pd.read_csv(r'C:\Users\Adnane\Desktop\Radar\ObjectClassificationWithRadar\Training_with_transfer_learning\spectrogram_features_google_vite.csv')

In [2]:
INSHEP

,file_id,activity,path,mean_entropy,mean_power,variance,stddev,max_vel,amp_density,kurtosis,...,mean_torso_power,pos_neg_ratio,doppler_offset,main_lobe_width,motion_duration,doppler_peak_velocity,doppler_symmetry_index,cepstral_entropy,range_bin_span,doppler_bandwidth
0,1P36A01R02,walking,C:\Users\Adnane\Desktop\Radar\ObjectClassifica...,9.025276,1.959870e+07,6.906348e+16,2.627993e+08,1.099138,0.000520,721.726235,...,7.179115e+07,1.530323,0.279423,0.258621,1.07,1.002155,0.209587,0.940833,6.0,0.836027
1,1P38A01R03,walking,C:\Users\Adnane\Desktop\Radar\ObjectClassifica...,9.258551,2.104924e+07,5.631026e+16,2.372978e+08,0.840517,0.000819,433.210982,...,6.311644e+07,1.251952,0.112505,0.161638,2.30,0.937500,0.111882,0.999645,6.0,0.915421
2,1P38A01R01,walking,C:\Users\Adnane\Desktop\Radar\ObjectClassifica...,9.140249,2.620588e+07,9.122603e+16,3.020365e+08,0.808190,0.000743,421.248890,...,1.433929e+08,1.574890,0.182513,0.193966,2.00,0.840517,0.223268,1.032866,5.0,0.705764
3,1P37A01R02,walking,C:\Users\Adnane\Desktop\Radar\ObjectClassifica...,9.187234,2.411605e+07,7.596749e+16,2.756220e+08,0.840517,0.000945,389.177448,...,1.239555e+08,0.917316,-0.039091,1.907328,2.01,0.840517,-0.043125,0.982990,6.0,0.738319
4,1P36A01R03,walking,C:\Users\Adnane\Desktop\Radar\ObjectClassifica...,8.914113,1.945387e+07,7.617443e+16,2.759972e+08,0.969828,0.000692,700.990139,...,6.483077e+07,2.064459,0.364250,0.193966,1.37,0.905172,0.347356,0.921222,6.0,0.762972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1749,5P55A05R2,drink_water,C:\Users\Adnane\Desktop\Radar\ObjectClassifica...,7.935626,5.718070e+05,7.192478e+13,8.480848e+06,0.161638,0.000559,640.340498,...,2.337248e+07,0.927633,-0.015329,0.420259,0.42,0.161638,-0.037542,0.653246,3.0,0.272728
1750,5P55A05R3,drink_water,C:\Users\Adnane\Desktop\Radar\ObjectClassifica...,8.041883,9.940976e+05,1.895267e+14,1.376687e+07,-0.193966,0.000572,528.732149,...,3.984330e+07,1.403761,0.040513,0.387931,0.59,0.129310,0.167971,0.877308,2.0,0.242563
1751,5P56A05R1,drink_water,C:\Users\Adnane\Desktop\Radar\ObjectClassifica...,7.231323,2.052530e+06,1.951923e+15,4.418057e+07,0.161638,0.000533,1119.092028,...,8.719889e+07,1.275885,0.032924,0.420259,0.46,0.161638,0.121221,1.052820,2.0,0.204817
1752,5P56A05R2,drink_water,C:\Users\Adnane\Desktop\Radar\ObjectClassifica...,7.295752,1.713943e+06,1.355906e+15,3.682262e+07,0.193966,0.000288,1515.745649,...,6.577793e+07,1.193892,0.027241,0.484914,0.26,0.193966,0.088378,1.140604,2.0,0.228499


In [3]:
# First, extract the folder name from the Windows-style path
INSHEP['folder'] = INSHEP['path'].str.extract(r'datasets\\([^\\]+)\\')

# Then, build the full image path
INSHEP['image_path'] = 'spectrograms/' + INSHEP['folder'] + '/' + INSHEP['file_id'] + '_spectrogram.png'

# Optional: drop the helper 'folder' column
INSHEP.drop(columns=['folder'], inplace=True)


In [4]:
INSHEP
#display where path is duplicated and order by path
INSHEP[INSHEP.duplicated(subset=['path'], keep=False)].sort_values(by='path')

#remove duplicates based on 'path' column, keeping the first occurrence
INSHEP = INSHEP.drop_duplicates(subset=['path'], keep='first')
INSHEP

,file_id,activity,path,mean_entropy,mean_power,variance,stddev,max_vel,amp_density,kurtosis,...,pos_neg_ratio,doppler_offset,main_lobe_width,motion_duration,doppler_peak_velocity,doppler_symmetry_index,cepstral_entropy,range_bin_span,doppler_bandwidth,image_path
0,1P36A01R02,walking,C:\Users\Adnane\Desktop\Radar\ObjectClassifica...,9.025276,1.959870e+07,6.906348e+16,2.627993e+08,1.099138,0.000520,721.726235,...,1.530323,0.279423,0.258621,1.07,1.002155,0.209587,0.940833,6.0,0.836027,spectrograms/1 December 2017 Dataset/1P36A01R0...
1,1P38A01R03,walking,C:\Users\Adnane\Desktop\Radar\ObjectClassifica...,9.258551,2.104924e+07,5.631026e+16,2.372978e+08,0.840517,0.000819,433.210982,...,1.251952,0.112505,0.161638,2.30,0.937500,0.111882,0.999645,6.0,0.915421,spectrograms/1 December 2017 Dataset/1P38A01R0...
2,1P38A01R01,walking,C:\Users\Adnane\Desktop\Radar\ObjectClassifica...,9.140249,2.620588e+07,9.122603e+16,3.020365e+08,0.808190,0.000743,421.248890,...,1.574890,0.182513,0.193966,2.00,0.840517,0.223268,1.032866,5.0,0.705764,spectrograms/1 December 2017 Dataset/1P38A01R0...
3,1P37A01R02,walking,C:\Users\Adnane\Desktop\Radar\ObjectClassifica...,9.187234,2.411605e+07,7.596749e+16,2.756220e+08,0.840517,0.000945,389.177448,...,0.917316,-0.039091,1.907328,2.01,0.840517,-0.043125,0.982990,6.0,0.738319,spectrograms/1 December 2017 Dataset/1P37A01R0...
4,1P36A01R03,walking,C:\Users\Adnane\Desktop\Radar\ObjectClassifica...,8.914113,1.945387e+07,7.617443e+16,2.759972e+08,0.969828,0.000692,700.990139,...,2.064459,0.364250,0.193966,1.37,0.905172,0.347356,0.921222,6.0,0.762972,spectrograms/1 December 2017 Dataset/1P36A01R0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1749,5P55A05R2,drink_water,C:\Users\Adnane\Desktop\Radar\ObjectClassifica...,7.935626,5.718070e+05,7.192478e+13,8.480848e+06,0.161638,0.000559,640.340498,...,0.927633,-0.015329,0.420259,0.42,0.161638,-0.037542,0.653246,3.0,0.272728,spectrograms/7 March 2019 West Cumbria Dataset...
1750,5P55A05R3,drink_water,C:\Users\Adnane\Desktop\Radar\ObjectClassifica...,8.041883,9.940976e+05,1.895267e+14,1.376687e+07,-0.193966,0.000572,528.732149,...,1.403761,0.040513,0.387931,0.59,0.129310,0.167971,0.877308,2.0,0.242563,spectrograms/7 March 2019 West Cumbria Dataset...
1751,5P56A05R1,drink_water,C:\Users\Adnane\Desktop\Radar\ObjectClassifica...,7.231323,2.052530e+06,1.951923e+15,4.418057e+07,0.161638,0.000533,1119.092028,...,1.275885,0.032924,0.420259,0.46,0.161638,0.121221,1.052820,2.0,0.204817,spectrograms/7 March 2019 West Cumbria Dataset...
1752,5P56A05R2,drink_water,C:\Users\Adnane\Desktop\Radar\ObjectClassifica...,7.295752,1.713943e+06,1.355906e+15,3.682262e+07,0.193966,0.000288,1515.745649,...,1.193892,0.027241,0.484914,0.26,0.193966,0.088378,1.140604,2.0,0.228499,spectrograms/7 March 2019 West Cumbria Dataset...


In [5]:
merged_df = SPECTROGRAM.merge(INSHEP, on='image_path', how='inner')  # or 'left'/'right'/'outer' as needed

In [6]:
merged_df

,image_path,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,mean_torso_power,pos_neg_ratio,doppler_offset,main_lobe_width,motion_duration,doppler_peak_velocity,doppler_symmetry_index,cepstral_entropy,range_bin_span,doppler_bandwidth
0,spectrograms/4 July 2018 Dataset/3P66A03R2_spe...,-0.048340,0.055817,-0.145508,-0.373291,-0.008018,-0.236084,0.068909,0.140747,-0.217529,...,6.303265e+07,0.151130,-0.231972,0.161638,0.31,-0.193966,-0.737423,1.083437,6.0,0.236668
1,spectrograms/4 July 2018 Dataset/1P68A01R1_spe...,0.034607,-0.034241,-0.101257,-0.314209,-0.255371,-0.125732,0.016068,-0.000095,-0.256104,...,1.235783e+07,0.558474,-0.234554,0.420259,0.53,-0.872845,-0.283306,0.909129,12.0,0.832668
2,spectrograms/4 July 2018 Dataset/5P60A05R1_spe...,-0.086121,0.089172,-0.174683,-0.426758,-0.082092,-0.114990,-0.052063,0.198975,-0.243286,...,1.745773e+06,1.047543,0.001466,0.323276,0.05,0.129310,0.023219,0.410934,8.0,0.762034
3,spectrograms/4 July 2018 Dataset/3P65A03R3_spe...,-0.034821,0.034851,-0.234253,-0.427246,-0.048126,-0.329834,0.031830,0.111877,-0.276123,...,1.134535e+08,0.047521,-0.251409,0.129310,0.13,-0.161638,-0.909270,1.122153,6.0,0.204580
4,spectrograms/4 July 2018 Dataset/1P62A01R2_spe...,0.058044,-0.131958,-0.123047,-0.252197,-0.213013,-0.140991,0.030991,-0.042145,-0.204224,...,9.166222e+06,1.839353,0.272753,0.258621,0.93,0.905172,0.295614,0.826051,10.0,0.963742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1749,spectrograms/1 December 2017 Dataset/6P56A06R0...,-0.045807,-0.061523,-0.045197,-0.329102,-0.037079,-0.083801,-0.027954,0.187988,-0.169800,...,3.642903e+06,0.016558,-0.585832,0.193966,0.14,-0.517241,-0.967423,1.140569,5.0,0.293774
1750,spectrograms/1 December 2017 Dataset/1P42A01R0...,0.025467,-0.013214,-0.195801,-0.453125,-0.275391,-0.149902,-0.021835,0.052521,-0.262939,...,1.031794e+08,0.648905,-0.094934,1.745690,2.90,-0.743534,-0.212926,0.960106,6.0,0.736956
1751,spectrograms/1 December 2017 Dataset/6P40A06R0...,-0.044037,-0.016708,-0.088562,-0.362061,-0.119751,-0.167114,-0.026672,0.211792,-0.144775,...,1.974422e+07,0.014511,-0.433081,0.129310,0.31,-0.355603,-0.971394,1.054593,6.0,0.330681
1752,spectrograms/1 December 2017 Dataset/2P56A02R0...,0.064026,-0.040649,-0.171021,-0.335693,-0.171387,-0.112244,-0.003649,0.106323,-0.143188,...,1.168200e+08,15.373221,0.264797,0.161638,0.25,0.161638,0.877849,1.310697,5.0,0.270219


In [7]:
#encode the activity column like this 
ACTIVITY_MAP = {
    "walking": 1,
    "sitting_down": 2,
    "standing_up": 3,
    "pick_object": 4,
    "drink_water": 5,
    "fall": 6,
}
merged_df['activity'] = merged_df['activity'].map(ACTIVITY_MAP)

merged_df=merged_df.drop(columns=['path', 'file_id', 'image_path'])  # drop unnecessary columns
#split in train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(merged_df.drop(columns=['activity']),
                                                    merged_df['activity'],
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=merged_df['activity'])
#validation set
X_train, X_val, y_train, y_val = train_test_split(X_train,
                                                  y_train,
                                                  test_size=0.2,
                                                  random_state=42,
                                                  stratify=y_train)

In [8]:
#randomforest classifier
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
#fit the model
rf.fit(X_train, y_train)

#evaluate the model
from sklearn.metrics import classification_report, accuracy_score
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=ACTIVITY_MAP.keys()))
print("Accuracy:", accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

     walking       1.00      0.98      0.99        62
sitting_down       0.95      0.97      0.96        63
 standing_up       0.92      0.95      0.94        62
 pick_object       0.73      0.77      0.75        62
 drink_water       0.79      0.74      0.77        62
        fall       0.97      0.93      0.95        40

    accuracy                           0.89       351
   macro avg       0.89      0.89      0.89       351
weighted avg       0.89      0.89      0.89       351

Accuracy: 0.8888888888888888


In [12]:
#standardize the features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)
#fit the model again with scaled features
rf.fit(X_train_scaled, y_train)
#evaluate the model again
y_pred_scaled = rf.predict(X_test_scaled)
print(classification_report(y_test, y_pred_scaled))
print("Accuracy with scaled features:", accuracy_score(y_test, y_pred_scaled))

              precision    recall  f1-score   support

           1       1.00      0.98      0.99        62
           2       0.95      0.97      0.96        63
           3       0.92      0.95      0.94        62
           4       0.73      0.77      0.75        62
           5       0.79      0.74      0.77        62
           6       0.97      0.93      0.95        40

    accuracy                           0.89       351
   macro avg       0.89      0.89      0.89       351
weighted avg       0.89      0.89      0.89       351

Accuracy with scaled features: 0.8888888888888888


In [13]:
#PCA for dimensionality reduction
from sklearn.decomposition import PCA
pca = PCA(n_components=30)  # retain 95% of variance
X_train_pca = pca.fit_transform(X_train_scaled)
X_val_pca = pca.transform(X_val_scaled)
X_test_pca = pca.transform(X_test_scaled)
#fit the model again with PCA features
rf.fit(X_train_pca, y_train)
#evaluate the model again
y_pred_pca = rf.predict(X_test_pca)
print(classification_report(y_test, y_pred_pca))
print("Accuracy with PCA features:", accuracy_score(y_test, y_pred_pca))


              precision    recall  f1-score   support

           1       1.00      0.97      0.98        62
           2       0.83      0.78      0.80        63
           3       0.72      0.77      0.74        62
           4       0.65      0.68      0.66        62
           5       0.66      0.68      0.67        62
           6       0.94      0.85      0.89        40

    accuracy                           0.78       351
   macro avg       0.80      0.79      0.79       351
weighted avg       0.79      0.78      0.79       351

Accuracy with PCA features: 0.7834757834757835


In [16]:
#leave one out feature search
from sklearn.feature_selection import RFECV
selector = RFECV(estimator=rf, step=1, cv=5, scoring='accuracy', verbose = 4, n_jobs=-1)
selector = selector.fit(X_train, y_train)
# Get the selected features
selected_features = X_train.columns[selector.support_]
print("Selected features:", selected_features.tolist())


Fitting estimator with 787 features.
Fitting estimator with 786 features.
Fitting estimator with 785 features.
Fitting estimator with 784 features.
Fitting estimator with 783 features.
Fitting estimator with 782 features.
Fitting estimator with 781 features.
Fitting estimator with 780 features.
Fitting estimator with 779 features.
Fitting estimator with 778 features.
Fitting estimator with 777 features.
Fitting estimator with 776 features.
Fitting estimator with 775 features.
Fitting estimator with 774 features.
Fitting estimator with 773 features.
Fitting estimator with 772 features.
Fitting estimator with 771 features.
Fitting estimator with 770 features.
Fitting estimator with 769 features.
Fitting estimator with 768 features.
Fitting estimator with 767 features.
Fitting estimator with 766 features.
Fitting estimator with 765 features.
Fitting estimator with 764 features.
Fitting estimator with 763 features.
Fitting estimator with 762 features.
Fitting estimator with 761 features.
F

In [17]:
#only keep selected features
#Selected features: ['feature_2', 'feature_8', 'feature_13', 'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_27', 'feature_29', 'feature_34', 'feature_40', 'feature_41', 'feature_50', 'feature_61', 'feature_62', 'feature_64', 'feature_66', 'feature_67', 'feature_72', 'feature_79', 'feature_82', 'feature_84', 'feature_87', 'feature_89', 'feature_90', 'feature_92', 'feature_108', 'feature_114', 'feature_116', 'feature_120', 'feature_127', 'feature_128', 'feature_132', 'feature_136', 'feature_137', 'feature_138', 'feature_141', 'feature_150', 'feature_151', 'feature_157', 'feature_159', 'feature_164', 'feature_165', 'feature_166', 'feature_170', 'feature_172', 'feature_176', 'feature_180', 'feature_182', 'feature_184', 'feature_185', 'feature_186', 'feature_190', 'feature_192', 'feature_198', 'feature_204', 'feature_205', 'feature_210', 'feature_213', 'feature_214', 'feature_221', 'feature_225', 'feature_229', 'feature_233', 'feature_234', 'feature_240', 'feature_243', 'feature_254', 'feature_256', 'feature_257', 'feature_262', 'feature_266', 'feature_267', 'feature_269', 'feature_270', 'feature_275', 'feature_276', 'feature_278', 'feature_279', 'feature_284', 'feature_288', 'feature_293', 'feature_297', 'feature_298', 'feature_301', 'feature_306', 'feature_308', 'feature_309', 'feature_311', 'feature_314', 'feature_315', 'feature_319', 'feature_324', 'feature_328', 'feature_331', 'feature_334', 'feature_336', 'feature_337', 'feature_340', 'feature_342', 'feature_344', 'feature_353', 'feature_356', 'feature_357', 'feature_367', 'feature_371', 'feature_375', 'feature_377', 'feature_380', 'feature_381', 'feature_383', 'feature_385', 'feature_387', 'feature_389', 'feature_390', 'feature_395', 'feature_396', 'feature_407', 'feature_408', 'feature_409', 'feature_413', 'feature_415', 'feature_418', 'feature_426', 'feature_427', 'feature_430', 'feature_432', 'feature_441', 'feature_444', 'feature_448', 'feature_452', 'feature_462', 'feature_465', 'feature_466', 'feature_467', 'feature_477', 'feature_479', 'feature_483', 'feature_486', 'feature_489', 'feature_492', 'feature_497', 'feature_498', 'feature_500', 'feature_504', 'feature_512', 'feature_515', 'feature_521', 'feature_524', 'feature_527', 'feature_538', 'feature_541', 'feature_544', 'feature_547', 'feature_550', 'feature_552', 'feature_556', 'feature_557', 'feature_558', 'feature_564', 'feature_565', 'feature_570', 'feature_574', 'feature_579', 'feature_580', 'feature_581', 'feature_582', 'feature_584', 'feature_587', 'feature_589', 'feature_596', 'feature_597', 'feature_605', 'feature_614', 'feature_617', 'feature_620', 'feature_624', 'feature_629', 'feature_630', 'feature_639', 'feature_650', 'feature_651', 'feature_654', 'feature_656', 'feature_665', 'feature_667', 'feature_668', 'feature_671', 'feature_673', 'feature_679', 'feature_685', 'feature_690', 'feature_695', 'feature_705', 'feature_712', 'feature_714', 'feature_718', 'feature_719', 'feature_724', 'feature_732', 'feature_735', 'feature_737', 'feature_741', 'feature_747', 'feature_748', 'feature_753', 'feature_756', 'feature_760', 'feature_765', 'feature_766', 'mean_entropy', 'mean_power', 'variance', 'stddev', 'max_vel', 'periodicity', 'mean_torso_power', 'pos_neg_ratio', 'doppler_offset', 'main_lobe_width', 'doppler_peak_velocity', 'doppler_symmetry_index', 'cepstral_entropy', 'doppler_bandwidth']

X_train = X_train[selected_features]
X_test = X_test[selected_features]
X_val = X_val[selected_features]

In [20]:
#random forest classifier with selected features
rf.fit(X_train, y_train)
#evaluate the model with selected features
y_pred_selected = rf.predict(X_test)
print(classification_report(y_test, y_pred_selected))
print("Accuracy with selected features:", accuracy_score(y_test, y_pred_selected))


              precision    recall  f1-score   support

           1       1.00      0.98      0.99        62
           2       0.95      0.97      0.96        63
           3       0.91      0.94      0.92        62
           4       0.72      0.77      0.74        62
           5       0.79      0.73      0.76        62
           6       0.97      0.93      0.95        40

    accuracy                           0.88       351
   macro avg       0.89      0.89      0.89       351
weighted avg       0.88      0.88      0.88       351

Accuracy with selected features: 0.8831908831908832


In [18]:
#MLP classifier
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
#fit the model
mlp.fit(X_train_scaled, y_train)
#evaluate the model
y_pred_mlp = mlp.predict(X_test_scaled)
print(classification_report(y_test, y_pred_mlp))
print("MLP Accuracy:", accuracy_score(y_test, y_pred_mlp))

              precision    recall  f1-score   support

           1       1.00      0.98      0.99        62
           2       0.93      0.98      0.95        63
           3       0.97      0.97      0.97        62
           4       0.69      0.79      0.74        62
           5       0.78      0.65      0.71        62
           6       0.97      0.95      0.96        40

    accuracy                           0.88       351
   macro avg       0.89      0.89      0.89       351
weighted avg       0.89      0.88      0.88       351

MLP Accuracy: 0.8831908831908832


In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd # Assuming X_train_pca is a pandas DataFrame
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split



# Encode labels to start from 0 using LabelEncoder
le = LabelEncoder()
# Fit on all unique labels across all sets to ensure consistency
le.fit(np.unique(np.concatenate([y_train, y_val, y_test])))

# Transform all label sets
y_train_tensor = torch.tensor(le.transform(y_train), dtype=torch.long)
y_val_tensor = torch.tensor(le.transform(y_val), dtype=torch.long)
y_test_tensor = torch.tensor(le.transform(y_test), dtype=torch.long)

# Define number of classes
num_classes = len(le.classes_)

# Debug: Check label ranges
print("Unique values in y_train_tensor:", torch.unique(y_train_tensor))
print("Unique values in y_val_tensor:", torch.unique(y_val_tensor))
print("Unique values in y_test_tensor:", torch.unique(y_test_tensor))
print("num_classes:", num_classes)

# Assertions for label range (already present, good practice)
assert torch.all(y_train_tensor >= 0) and torch.all(y_train_tensor < num_classes), \
    f"y_train_tensor has out-of-range labels. Min: {y_train_tensor.min()}, Max: {y_train_tensor.max()}, Expected range [0, {num_classes-1}]"
assert torch.all(y_val_tensor >= 0) and torch.all(y_val_tensor < num_classes), \
    f"y_val_tensor has out-of-range labels. Min: {y_val_tensor.min()}, Max: {y_val_tensor.max()}, Expected range [0, {num_classes-1}]"
assert torch.all(y_test_tensor >= 0) and torch.all(y_test_tensor < num_classes), \
    f"y_test_tensor has out-of-range labels. Min: {y_test_tensor.min()}, Max: {y_test_tensor.max()}, Expected range [0, {num_classes-1}]"


# Convert PCA features to PyTorch tensors
# Use directly if X_pca is a numpy array
X_train_tensor = torch.tensor(X_train_pca, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_pca, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_pca, dtype=torch.float32)

# Check for NaNs/Infs in input tensors
print("X_train_tensor has NaNs:", torch.isnan(X_train_tensor).any())
print("X_train_tensor has Infs:", torch.isinf(X_train_tensor).any())
print("X_val_tensor has NaNs:", torch.isnan(X_val_tensor).any())
print("X_val_tensor has Infs:", torch.isinf(X_val_tensor).any())
print("X_test_tensor has NaNs:", torch.isnan(X_test_tensor).any())
print("X_test_tensor has Infs:", torch.isinf(X_test_tensor).any())


# Create DataLoader for training and validation sets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Define the neural network architecture
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.4)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x) # No activation here, as CrossEntropyLoss expects logits
        return x

# Initialize model, loss function, and optimizer
input_size = X_train_pca.shape[1]
model = SimpleNN(input_size, num_classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop with early stopping
num_epochs = 5000
patience = 50
best_val_loss = float('inf')
early_stopping_counter = 0
best_model_path = 'best_model.pth' # Define path for saving best model

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)

    # Validation step
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)

    val_loss /= len(val_loader.dataset)

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}')

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stopping_counter = 0
        torch.save(model.state_dict(), best_model_path)  # Save best model
    else:
        early_stopping_counter += 1

    if early_stopping_counter >= patience:
        print("Early stopping triggered")
        break

# Load best model for evaluation
print(f"Loading best model from {best_model_path}")
model.load_state_dict(torch.load(best_model_path))

# Evaluate on the test set
model.eval()
test_loss = 0.0
test_correct = 0
with torch.no_grad():
    test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=32)
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        test_correct += (predicted == labels).sum().item()

test_loss /= len(X_test_tensor)
test_accuracy = test_correct / len(X_test_tensor)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

# Save final model (optional, often you'd just use the best_model.pth for deployment)
final_model_path = 'final_model.pth'
torch.save(model.state_dict(), final_model_path)
print(f"Final model state saved to {final_model_path}")

# Load model for inference (example)
# loaded_model = SimpleNN(input_size, num_classes)
# loaded_model.load_state_dict(torch.load('final_model.pth'))
# loaded_model.to(device) # Don't forget to move it to device if you want to use it on GPU
# loaded_model.eval()

Unique values in y_train_tensor: tensor([0, 1, 2, 3, 4, 5])
Unique values in y_val_tensor: tensor([0, 1, 2, 3, 4, 5])
Unique values in y_test_tensor: tensor([0, 1, 2, 3, 4, 5])
num_classes: 6
X_train_tensor has NaNs: tensor(False)
X_train_tensor has Infs: tensor(False)
X_val_tensor has NaNs: tensor(False)
X_val_tensor has Infs: tensor(False)
X_test_tensor has NaNs: tensor(False)
X_test_tensor has Infs: tensor(False)
Using device: cuda
Epoch [1/5000], Loss: 1.1320, Val Loss: 0.6731
Epoch [2/5000], Loss: 0.6543, Val Loss: 0.5355
Epoch [3/5000], Loss: 0.5524, Val Loss: 0.4800
Epoch [4/5000], Loss: 0.4963, Val Loss: 0.4581
Epoch [5/5000], Loss: 0.4494, Val Loss: 0.4516
Epoch [6/5000], Loss: 0.4242, Val Loss: 0.4367
Epoch [7/5000], Loss: 0.3950, Val Loss: 0.4580
Epoch [8/5000], Loss: 0.3736, Val Loss: 0.4286
Epoch [9/5000], Loss: 0.3287, Val Loss: 0.4390
Epoch [10/5000], Loss: 0.3333, Val Loss: 0.4255
Epoch [11/5000], Loss: 0.3216, Val Loss: 0.4572
Epoch [12/5000], Loss: 0.3122, Val Loss: 0

C:\Users\Adnane\AppData\Local\Temp\ipykernel_24760\3199207767.py:139: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))
